# 利用llama_index+ipex_llm快速开发基于RAG技术的笔记本产品说明书和电脑配置助手

### 安装依赖 

In [ ]:
!pip install -r requirements.txt

### 加载分词模型

In [2]:
from transformers import AutoTokenizer

# 加载本地模型
tokenizer_model_id_or_path = r"D:\models\Qwen\Qwen2___5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_id_or_path)


In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.llama_cpp import LlamaCPP

model_path = r"D:\models\Qwen\Qwen2___5-7B-Instruct-GGUF\qwen2.5-7b-instruct-q2_k.gguf"

llm = LlamaCPP(
    model_url=None,
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=512,
    verbose=True,
)
Settings.llm = llm

llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from D:\models\Qwen\Qwen2___5-7B-Instruct-GGUF\qwen2.5-7b-instruct-q2_k.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = qwen2.5-7b-instruct
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                           general.finetune str              = qwen2.5-7b-instruct
llama_model_loader: - kv   5:                         general.size_label str              = 7.6B
llama_model_loader: - kv   6:                          qwen2.block_count u32              = 28
llama_model_

## 接下来，使用增强检索

### 使用set_global_tokenizer配置分词模型

In [4]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

tokenizer_model_id_or_path = r"D:\models\Qwen\Qwen2___5-7B-Instruct"
set_global_tokenizer(
    AutoTokenizer.from_pretrained(tokenizer_model_id_or_path).encode
)

### 使用HuggingFaceEmbedding配置embed模型

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_path = r"D:\models\BAAI\bge-m3"
embed_model = HuggingFaceEmbedding(model_name=embed_model_path)
Settings.embed_model = embed_model

### 加载PDF

In [6]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("./data").load_data()

In [7]:
# 查看前两端文档内容
for doc in documents[:2]:
    print(f"文档 ID: {doc.doc_id}")
    print(f"文档内容: {doc.text}")
    print("-" * 50)  # 分隔不同的文档

文档 ID: f498ee9c-f075-4608-b46e-a2ed445a5500
文档内容: P/N: 147003835
© 版权所有 联想 2010
Lenovo
安全及通用信息指南
www.lenovo.com.cn
New World. New Thinking.
TM
--------------------------------------------------
文档 ID: 4e126f84-5fb9-4eab-bbc4-9ab24bab1a17
文档内容: 
--------------------------------------------------


In [8]:
# 查看文档内容
for doc in documents[-2:]:
    print(f"文档 ID: {doc.doc_id}")
    print(f"文档内容: {doc.text}")
    print("-" * 50)  # 分隔不同的文档

文档 ID: 8982bf3f-abf0-41a4-9b57-dfbd6cb8cc53
文档内容: 23. 联想 IdeaPad 3 Gen 8：AMD Ryzen 5 7535U，8GB DDR4，512GB SSD，15.6 英
寸 FHD 屏，续航可达 10 小时 3。 
24. 联想 IdeaPad Flex 5i：Intel Core i5 1335U，16GB LPDDR4X，1TB SSD，14 英
寸 FHD 触控屏，续航可达 12 小时 3。 
25. 联想 Yoga 9i Gen 8：Intel Core i9 13900H，32GB LPDDR5，2TB PCIe NVMe SSD，
14 英寸 4K OLED 触控屏，续航可达 14 小时 3。 
26. 联想 Legion 5 Pro：AMD Ryzen 7 5800H，16GB DDR4，1TB SSD，NVIDIA GeForce 
RTX 3060 显卡，16 英寸 QHD、165Hz 屏，续航可达 8 小时 3。 
27. 联想 ThinkPad X390 ：英特尔酷睿  i5 或 i7 处理器， 8GB 或 16GB 内存，
512GB 或 1TB SSD， 集成显卡，13.3 英寸高分辨率屏幕， 适合经常出差人士2。 
28. 联想 ThinkPad X395：AMD 锐龙处理器，8GB 或 16GB 内存，512GB 或 1TB 
SSD，集成显卡，13.3 英寸高分辨率屏幕，轻薄便携，续航出色 2。 
29. 联想 Yoga C940：英特尔酷睿 i7 或 i9 处理器，16GB 或 32GB 内存，512GB 
或 1TB SSD，4K 触控屏，杜比全景声音响系统，支持 360 度翻转 2。 
30. 联想 Yoga S740： 英特尔酷睿 i7 处理器，16GB 内存，512GB 或 1TB SSD，高
分辨率屏幕，长效电池，适合日常办公娱乐 2。 
31. 联想 Yoga Duet 7i： 英特尔处理器，8GB 或 16GB 内存，256GB 或 512GB SSD，
可拆卸键盘，支持手写笔输入，高分辨率触控屏 2。 
32. 联想 Legion Y740： 英特尔酷睿 i7 处理器，16GB 内存，512GB 或 1TB SSD，
NVIDIA GeForce RTX

### 生成向量引擎

In [9]:
from llama_index.core import VectorStoreIndex

# create vector store index
index = VectorStoreIndex.from_documents(documents)

### 输出检索结果

In [10]:
# 创建检索器
retriever = index.as_retriever(
    similarity_top_k=3,  # 返回最相似的3个文档
    search_kwargs={"score_threshold": 0.7},# 相似度阈值,
    document_content_description="关于联想电脑的产品说明书", # 文档内容描述
)


In [11]:
# 直接使用查询引擎获取答案
query_engine = index.as_query_engine()
response = query_engine.query("如果我想购买14英寸的电脑有哪些？")

llama_perf_context_print:        load time =  188345.44 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  1761 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   511 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  320181.48 ms /  2272 tokens


In [12]:
print(response)

14英寸的电脑有以下几种选择：

1. 联想 ideapad flex5：配备AMD锐龙R5或R7处理器，8GB或16GB内存，512GB SSD，集成AMD Radeon Graphics显卡，14英寸IPS触控屏。
2. 联想小新 Air15：配备英特尔酷睿i5或i7处理器，8GB或16GB内存，512GB SSD，集成或独立显卡，15.6英寸屏幕。
3. 联想 YOGA 9i 14 款：配备英特尔酷睿i7处理器，16GB内存，512GB SSD，14英寸4K触摸屏，支持360度翻转。
4. 联想  ThinkPad X1 Carbon ：配备英特尔酷睿i5或i7处理器，8GB或16GB LPDDR4X内存，512GB或1TB SSD，集成英特尔 UHD Graphics 显卡，14英寸高分辨率屏幕。
5. 联想 ThinkPad T14：配备英特尔酷睿i5或i7处理器，8GB或16GB DDR4内存，512GB或1TB SSD，集成或独立显卡，14英寸IPS屏。
6. 联想小新 14 2024：配备酷睿i5-13420H，16GB内存，512GB SSD，集成显卡，14英寸屏幕。
7. 联想小新 Pro14 2024 AI 超能本：配备酷睿i5-13420H等，16GB内存，512GB或1TB SSD，集成显卡，14英寸2.8K屏幕。
8. 联想拯救者 Y7000P 2024：配备酷睿i7-14700HX，16GB内存，1TB SSD，显卡RTX4060或RTX4070，15.6英寸IPS屏，高刷新率。
9. 联想拯救者 R7000 2024：配备R7-8745H处理器，16GB内存，512GB SSD，RTX4060
